Imports

In [32]:
import numpy as np
import cv2
import os
import glob

import matplotlib.pyplot as plt

from cv2 import aruco

from calibration_utils import *

Setting Source directory for camera calibration

In [33]:
root = '/home/eryk-dev/Downloads/'

data_path = root + '/master/'

img_ext = "*.jpg"

img_shape = (3072, 4096)

Setting calibration values

In [34]:
board_shape = [8, 11]


# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


Preparing object points

In [35]:
dictionary = aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters =  aruco.DetectorParameters()
arucoDetector = aruco.ArucoDetector(dictionary, parameters)


board = aruco.CharucoBoard(board_shape, 0.044, 0.034, dictionary)

Flag to display images while collecting data.

In [36]:
display_images = False

Processing Images

In [ ]:

images = glob.glob(data_path + img_ext)
images.sort()

print(len(images))

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)

corner_ids = cornerIds(board_shape)

allCorners = []
allIds = []
decimator = 0

#objPoints, imgPoints = [], []


for i, fname in enumerate(images):
    print("[" + str((i+1)) + "/" + str(len(images)) + "]")
    head, tail = os.path.split(fname)

    bgr = cv2.imread(images[i], cv2.IMREAD_UNCHANGED)

    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    corners, ids, rejectedCorners = arucoDetector.detectMarkers(gray)

    if len(corners) > 16:
        for corner in corners:
            cv2.cornerSubPix(gray, corner,
                                winSize = (3,3),
                                zeroZone = (-1,-1),
                                criteria = criteria)
                    
        res2 = aruco.interpolateCornersCharuco(corners, ids, gray, board)
            
        if res2[1] is not None and res2[2] is not None and len(res2[1])>3 and decimator%1==0 and max(ids) <= max(board.getIds()):
            if is_slice_in_list(numpy.squeeze(ids).tolist(), corner_ids): # all corners are detected
                params, board_rot_deg  = get_parameters(corners, numpy.squeeze(ids).tolist(), corner_ids, img_shape, board_shape)
                
                allCorners.append(res2[1])
                allIds.append(res2[2])

                decimator+=1
                    
                print('Chessboard detected ' + tail)

                if display_images :
                    # Draw and display the corners
                    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
                    
                    frame_copy = aruco.drawDetectedMarkers(rgb, corners, ids)
                    
                    plt.imshow(frame_copy)
                    plt.show()

                    if(tail == '1690359127.2271936.jpg'):
                        frame_copy = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB)
                        cv2.imwrite('img.png', frame_copy)

        else:
            print("chessboards missing")

    else:
        print("chessboards missing")

print("\nDetected " + str(len(allCorners)) + " points")

Preparing for stereo calibration

In [38]:
stereocalib_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 1000, 1e-5)

In [39]:
print("CAMERA CALIBRATION")

cameraMatrixInit = np.array([[ 1000.,    0., img_shape[0] / 2.],
                            [    0., 1000.,  img_shape[1] / 2.],
                            [    0.,    0.,           1.]])

distCoeffsInit = np.zeros((5,1))

flags = 0#(cv2.CALIB_USE_INTRINSIC_GUESS + cv2.CALIB_RATIONAL_MODEL + cv2.CALIB_FIX_ASPECT_RATIO)
#flags = (cv2.CALIB_RATIONAL_MODEL)

CAMERA CALIBRATION


Stereo calibration

In [40]:
print("Calculating ....")


(ret, camera_matrix, distortion_coefficients,
     rotation_vectors, translation_vectors,
     stdDeviationsIntrinsics, stdDeviationsExtrinsics,
     perViewErrors) = cv2.aruco.calibrateCameraCharucoExtended(
                      charucoCorners=allCorners,
                      charucoIds=allIds,
                      board=board,
                      imageSize=img_shape,
                      cameraMatrix=cameraMatrixInit,
                      distCoeffs=distCoeffsInit,
                      flags=flags,
                      criteria=(cv2.TERM_CRITERIA_EPS & cv2.TERM_CRITERIA_COUNT, 10000, 1e-10))

print("Done")

Calculating ....
Done


Display Error

In [41]:
print("error: {}".format(ret))

error: 1.4977826829631329


Printing results

In [42]:
print('Intrinsic_mtx', camera_matrix)
print('dist', distortion_coefficients)

Intrinsic_mtx [[1.98825698e+03 0.00000000e+00 1.58478242e+03]
 [0.00000000e+00 2.00283028e+03 2.05003471e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist [[ 0.07146394]
 [-0.02555844]
 [-0.00151345]
 [ 0.00151731]
 [-0.0130729 ]]


Saving results

In [43]:
np.savetxt('Intrinsic_mtx_1.txt', camera_matrix)
np.savetxt('dist_1.txt', distortion_coefficients)
#np.savetxt('rotation_vectors.txt', rotation_vectors)
#np.savetxt('translation_vectors.txt', translation_vectors)

In [44]:
imgpoints, objpoints = calculateImgPointsObjPoints(allIds, allCorners, board)

In [45]:
#Reprojection error
def computeReprojectionErrors(objectPoints, imagePoints, rvecs, tvecs, cameraMatrix, distCoeffs, perViewErrors):
    imagePoints2 = []
    totalPoints = 0
    totalErr = 0

    for i in range(len(objectPoints)):
        imagePoints2, _ = cv2.projectPoints(objectPoints[i], rvecs[i], tvecs[i], cameraMatrix, distCoeffs)

        err = np.linalg.norm(np.array(imagePoints[i]) - np.array(imagePoints2))
        n = len(objectPoints[i])
        perViewErrors.append(np.sqrt(err**2 / n))
        totalErr += err**2
        totalPoints += n

    return np.sqrt(totalErr / totalPoints)

perViewErrors = []


err = computeReprojectionErrors(objpoints, imgpoints, rotation_vectors, translation_vectors, camera_matrix, distortion_coefficients, perViewErrors)


for (frame, err) in zip(images, perViewErrors):
    print(os.path.basename(frame) + ' ' + str(err))


1690359123.7718804.jpg 1.2989946317734045
1690359124.148325.jpg 0.9847858209558569
1690359126.3590202.jpg 1.3020947058250771
1690359127.2271936.jpg 2.3065492467835935
1690359127.6013777.jpg 1.991337246212442
1690359128.4469304.jpg 1.750098549948382
1690359132.8351154.jpg 1.863189473287032
1690359136.254239.jpg 0.7559575656420479
1690359137.0911386.jpg 2.5870995082576003
1690359137.4814637.jpg 1.7862311628758312
1690359140.6098385.jpg 1.2086005212051818
1690359141.7048762.jpg 0.9778880792504143
1690359142.7671614.jpg 1.0061929396555074
1690359143.8441384.jpg 1.020800116537068
1690359145.1489396.jpg 1.1308339946937451
1690359146.2468128.jpg 1.2794731815311875
1690359146.6226234.jpg 1.2467016703267844
1690359147.7036483.jpg 1.2723210233355684
1690359148.3016598.jpg 2.08114467144243
1690359149.3789983.jpg 2.5929048092632883
1690359153.9743443.jpg 0.8727740027697038
1690359155.0317051.jpg 1.3274661395389984
1690359155.8810792.jpg 2.348552816585163
1690359156.4902434.jpg 1.658408191922145
16